In [ ]:
#Imports

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab - DONE
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

#import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab -  DONE
import folium # map rendering library

from math import sin, cos, sqrt, atan2, radians



print('Libraries imported.')

Solving environment: \ 

### Introduction
The aim of this project is to enhance electric mobility in Barcelona as part of the IBM Capstone on Data Science. It locates the nearest electric parking stations near to a referenced food venue from Foursquare within Barcelona.
The project clusters Barcelona’s neighborhood based on the most common type of restaurants venues from Foursquare and to a selected place locates the nearest electric vehicle charging stations. 


In [ ]:
### Data readiness 

The initial data requires identifying and locating Barcelona’s neighborhood. The data used in this project has been downloaded from the Barcelona City Hall Open Data website. 
Once Barcelona’s neighborhoods have been listed, the list of addresses including street coordinates was used to generate the neighborhood coordinates as the mean of each one of its different addresses. Albeit it might not be the official neighborhood’s coordinates; it should be representative as an average among all their addresses.
Three different databases were loaded; two of them to compile the required information about Barcelona’s neighborhoods and the third include the reference to the electric charging stations in Barcelona which had the coordinates.

Data extractet from Barcelona city hall open data. 

Source: https://opendata-ajuntament.barcelona.cat/en/

1. Get neighborhoods list and addresses with coordinates.
2. Merge neighborhoods with coordinates mean

In [ ]:
### Load file with Barcelona neighborhoods
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_634f0334f89f49259087cb8a63af5ecf = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_BDh8zOunGUBK1B1_gxMeEDwKMmz6SlqEtCePSAMsRZY',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_634f0334f89f49259087cb8a63af5ecf.get_object(Bucket='pythonbasics-donotdelete-pr-m6t3t6sj4zu1mf',Key='2019_superficie.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_bcn = df_data_1[['Codi_Barri','Nom_Barri', 'Nom_Districte']]
df_bcn.set_index('Codi_Barri')

df_bcn.head()

In [ ]:
# Extract data from a stored csv file with BCN addresses
body = client_634f0334f89f49259087cb8a63af5ecf.get_object(Bucket='pythonbasics-donotdelete-pr-m6t3t6sj4zu1mf',Key='TAULA_DIRELE.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2 = df_data_2.groupby(['BARRI']).mean().reset_index()

df_coord = df_data_2[['BARRI','LATITUD', 'LONGITUD']]
#df_coord = df_coord.groupby(['BARRI']).mean()

#df_coord.set_index('BARRI')
df_coord.rename(columns={"BARRI":"Codi_Barri"}, inplace=True)

list(df_coord)
df_coord
               

In [ ]:
#Merge both dataframes
df_bcn = df_bcn.merge(df_coord)
df_bcn

In [ ]:
#Extract neighborhoods data
df_bcn.info()

In [ ]:
#Shape
df_bcn.shape

### Start mapping the data into map

Print map with neighborhoods

In [ ]:
#with open('geocoder_data.json') as json_data:
#    toronto_data = json.load(json_data)
url = 'https://geocoder.readthedocs.io/index.html'


!pip install geocoder
import geocoder # import geocoder



In [ ]:
def get_geocoder(postal_code_from_df):
    
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Barcelona, Spain'.format(postal_code_from_df))
        lat_lng_coords = g.latlng
        
    latitude = lat_lng_coords[0]
    print(latitude)
    longitude = lat_lng_coords[1]
    return latitude, longitude

### Filter by District

Center in barcelona and plot neighborhoods into a map

In [ ]:
#Center map in Barcelona
geolocator = Nominatim(user_agent="Barcelona")
location = geolocator.geocode('Barcelona')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona are {}, {}.'.format(latitude, longitude))



In [ ]:
### create map of Barcelona using latitude and longitude values
map_bcn = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_bcn['LATITUD'], df_bcn['LONGITUD'], df_bcn['Nom_Barri'], df_bcn['Nom_Districte']):
    label = '{}, {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bcn)  
    
map_bcn


### Foursquare account

1. Log in
2. Explore venues under food category
3. Review data - list of venues, classify per category and neigborhood


In [ ]:
# @hidden_cell
#Forusquare credentials
CLIENT_ID = 'BCVCI0ILGJW1T2WJMVB11ZH1UM22ISIEHZ4WEXFNK3WMBHQH' # your Foursquare ID
CLIENT_SECRET = '1WHDSKUWFXUCLRCJSNBW4TF22IGOOOCVQCOB2H12L3S3REHV' # your Foursquare Secret
VERSION = '20191225' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# defining radius and limit of venues to get
radius=500
LIMIT=100

In [ ]:
# limit research to restaurants (foof)
SECTION = "food"

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            SECTION,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# type your answer here

bcn_venues = getNearbyVenues(names=df_bcn['Nom_Barri'],
                                   latitudes=df_bcn['LATITUD'],
                                   longitudes=df_bcn['LONGITUD']
                                  )



In [ ]:
bcn_venues.head()

In [ ]:
bcn_venues.shape

In [ ]:
bcn_venues.groupby('Neighborhood').count()

In [ ]:
### TRIAL Analyze each neighborhood - ONLY RESTAURANTS
#bcn_venues.groupby('Venue Category').count()

#df[df['A'].str.contains("hello")]
#bcn_rest = bcn_venues[bcn_venues['Venue Category'].str.contains("Restaurant")]

In [ ]:


#one hot encoding
bcn_onehot = pd.get_dummies(bcn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bcn_onehot['Neighborhood'] = bcn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bcn_onehot.columns[-1]] + list(bcn_onehot.columns[:-1])
bcn_onehot = bcn_onehot[fixed_columns]

bcn_onehot.head()

In [ ]:
bcn_plot = bcn_venues.groupby('Venue Category').count().reset_index()

#Filter those over a min
num = len(bcn_plot['Neighborhood'])
suma = bcn_plot['Neighborhood'].sum()

#df.loc[df[‘column name’] condition]
bcn_plot_max = bcn_plot.loc[bcn_plot['Neighborhood'] >= 120]


num = len(bcn_plot_max['Neighborhood'])

#new_row = {'Venue Category':'Rest', 'Neighborhood':suma-num, 'Neighborhood Latitude':suma-num, 'Neighborhood Longitude' :suma-num, 'Venue':suma-num,'Venue Latitude':suma-num,'Venue Longitude':suma-num}
bcn_plot_max.loc[num]= ['Rest', suma-num, suma-num, suma-num, suma-num, suma-num, suma-num] #{'Venue Category':'Rest', 'Neighborhood':suma-num, 'Neighborhood Latitude':suma-num, 'Neighborhood Longitude' :suma-num, 'Venue':suma-num,'Venue Latitude':suma-num,'Venue Longitude':suma-num}
#bcn_plot_max = bcn_plot_max.append(new_row, ignore_index=True)
#bcn_plot_max


#bcn_plot_max.plot(kind = 'pie', y='Venue', shadow = False,startangle=90, figsize=(15,10), autopct='%1.1f%%', labels=bcn_plot_max['Venue Category'], title = 'Main Restaurant types')

title = plt.title('Main food category type', fontsize=15)
title.set_ha("left")

#fig = plt.figure(figsize=[10, 10])
#ax = fig.add_subplot(111)

plt.gca().axis("equal")

labels=bcn_plot_max['Venue Category']
labeldistance = 1.05

pie = plt.pie(bcn_plot_max['Venue'], startangle=90, autopct='%1.1f%%', radius =1.4, textprops={'fontsize': 10})
plt.legend(pie[0],labels, bbox_to_anchor=(1,0.5), loc="center right", fontsize=12, 
           bbox_transform=plt.gcf().transFigure)

plt.subplots_adjust(left=0.0, bottom=0.1, right=0.45)

plt.show()
bcn_plot_max


In [ ]:

#df = pd(bcn_onehot, columns=['Neighborhood','Bakery'])
#bcn_onehot.plot(x ='Neighborhood', y='Bakery', kind = 'bar')
#plt.show()

In [ ]:
### Group rows by neighborhood and by the mean of occurrence
bcn_grouped = bcn_onehot.groupby('Neighborhood').mean().reset_index()
bcn_grouped


In [ ]:
#### Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in bcn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bcn_grouped[bcn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
### Put in pandas dataframe


In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bcn_grouped['Neighborhood']

for ind in np.arange(bcn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bcn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted#.head()

In [ ]:
### Cluster neighborhoods

# set number of clusters
kclusters = 5

bcn_grouped_clustering = bcn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bcn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
#Run only once
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_) 

bcn_merged = df_bcn

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bcn_merged = bcn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Nom_Barri')

bcn_merged.tail() # check the last columns!

In [ ]:

#clean data
bcn_merged=bcn_merged.dropna()
#Cluster labels type
bcn_merged['Cluster_Labels'] = bcn_merged.Cluster_Labels.astype(int)


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bcn_merged['LATITUD'], bcn_merged['LONGITUD'], bcn_merged['Nom_Barri'], bcn_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
### Examine clusters

bcn_merged.loc[bcn_merged['Cluster_Labels'] == 0, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

In [ ]:
bcn_merged.loc[bcn_merged['Cluster_Labels'] == 1, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

In [ ]:
bcn_merged.loc[bcn_merged['Cluster_Labels'] == 2, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

In [ ]:
bcn_merged.loc[bcn_merged['Cluster_Labels'] == 3, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

In [ ]:
bcn_merged.loc[bcn_merged['Cluster_Labels'] == 4, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

### Electric car parking information

Load information related to car parking spots

In [ ]:
### Load data about electric car parking in BCN
body = client_634f0334f89f49259087cb8a63af5ecf.get_object(Bucket='pythonbasics-donotdelete-pr-m6t3t6sj4zu1mf',Key='PUNTS_RECARREGA_VEHICLES_ELECTRICS.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()

df_ecarpark = []
df_ecarpark = df_data_3[['TIPO_CONNECTOR','POTENCIA_MAXIMA_DE_CARGA_(W)', 'NUM_DE_CONNECTORS_AL_EQUIP', 'TIPOLOGIA_DE_CARREGA', 'LATITUD', 'LONGITUD']]
df_ecarpark = df_ecarpark.groupby(['LATITUD', 'LONGITUD'], as_index = False).agg({'TIPO_CONNECTOR': ','.join, 'POTENCIA_MAXIMA_DE_CARGA_(W)': max})
#df.groupby(['date'], as_index = False).agg({'muscles': ','.join})
df_ecarpark.head(5)

In [ ]:
#Measure distance between two data points

def get_distance(lat1, lon1, lat2, lon2):

    # approximate radius of earth in km

    R = 6373.0
    
   # for lat, lon in zip(lat2, lon2)

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [ ]:
#Select the reference 
#geopy.distance.vincenty(coords_1, coords_2).km

# La Vila de Gracia 41.403226	2.157236
lat1 = df_bcn.iloc[30][3]
lon1 = df_bcn.iloc[30][4]

df_dist = pd.DataFrame()
distance = []

# type your answer here

#for lat,lon in df_ecopark

# iterate through each row and select  
# 'Name' and 'Age' column respectively. 
for i in range(len(df_ecarpark)) : 
    dist = get_distance(lat1, lon1, df_ecarpark.loc[i, "LATITUD"], df_ecarpark.loc[i, "LONGITUD"]) 
    distance.append(dist)

df_ecarpark['DIST'] = distance 
   
df_ecarpark =df_ecarpark.sort_values(by='DIST')
df_parking = df_ecarpark.head(10)
df_parking.reset_index()

In [ ]:
# create map
map_parking = folium.Map(location=[lat1, lon1], zoom_start=12)

# add markers to the map
for lat, lon, conector, potencia_max in zip(df_parking['LATITUD'], df_parking['LONGITUD'], df_parking['TIPO_CONNECTOR'], df_parking['POTENCIA_MAXIMA_DE_CARGA_(W)']):
    label = '{}, {} W'.format(conector, potencia_max)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color='green',
        fill=True,
        #fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_parking)  

folium.Marker(
       [lat1, lon1],
        popup='La Vila de Gracia',# 41.403226	2.157236
        icon=folium.Icon(color='blue', icon='map-marker')
        ).add_to(map_parking)
    
map_parking